<a href="https://colab.research.google.com/github/mouha07/My-files/blob/main/Bert_imdb_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures
from sklearn.model_selection import train_test_split

In [35]:
# Chargement du jeu de données IMDB
dataImdb = pd.read_csv("/content/drive/MyDrive/MemoireM2/Notebook/Data_Imdb/IMDB Dataset.csv")
dataImdb.sample()

,review,sentiment
14708,Some of the greatest and most loved horror mov...,negative


In [6]:
# Installation de la bibliothèque Transformers
!pip install transformers

In [7]:
# Chargement du modèle BERT pré-entraîné et du tokenizer
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Transformation des étiquettes "positive" et "negative" en valeurs numériques (1 et 0)
def cat2num(value):
    if value == 'positive':
        return 1
    else:
        return 0

dataImdb['sentiment'] = dataImdb['sentiment'].apply(cat2num)

In [37]:
# Séparation des données en ensembles d'entraînement (70%) et de test + validation (30%)
train, test_val = train_test_split(dataImdb, test_size=0.3, random_state=42)

# Séparation des données de test_val en ensembles de test (50%) et de validation (50%)
test, validation = train_test_split(test_val, test_size=0.5, random_state=42)

In [38]:
# Enregistrement des données dans des fichiers CSV
train.to_csv("/content/drive/MyDrive/MemoireM2/Notebook/Data_Imdb/train.csv", index=False)
test.to_csv("/content/drive/MyDrive/MemoireM2/Notebook/Data_Imdb/test.csv", index=False)
validation.to_csv("/content/drive/MyDrive/MemoireM2/Notebook/Data_Imdb/validation.csv", index=False)

In [39]:
# Fonction pour convertir les exemples de données en objets InputExample
def convert2inputexamples(data_input, review, sentiment):
    data_examples = train.apply(lambda x: InputExample(guid=None, text_a=x[review], label=x[sentiment]), axis=1)
    return data_examples

train_examples= convert2inputexamples(train, 'review', 'sentiment')
validation_examples= convert2inputexamples(validation, 'review', 'sentiment')
test_examples= convert2inputexamples(test, 'review', 'sentiment')



In [18]:
train_examples.head(5)

38094    InputExample(guid=None, text_a='As much as I l...
40624    InputExample(guid=None, text_a="This was a ver...
49425    InputExample(guid=None, text_a='Not finding th...
35734    InputExample(guid=None, text_a='I\'m really su...
41708    InputExample(guid=None, text_a="I'll start by ...
dtype: object

In [40]:
validation.head(100)

,review,sentiment
45040,"The biggest National Lampoon hit remains ""Anim...",0
20713,"Eleven ""great"" filmmakers, eleven pieces of ga...",0
9836,I was intrigued by the nasty boss character as...,0
44734,"""Problem Child"" is one of the goofiest movies ...",0
40160,It is a shame that a movie with such a good ci...,0
...,...,...
11583,"In 17th Century Japan, there lived a samurai w...",0
2276,If you need cheering up on a cold weekday even...,1
15665,As I'm listening to my parents watch this (aft...,0
46078,What can you expect from a direct to DVD film?...,0


In [41]:
# Conversion des exemples en objets TF pour l'entraînement
def convertexamples2tf(examples, tokenizer, max_length=128):
    features = []

    for i in tqdm(examples):
        input_dict = tokenizer.encode_plus(
            i.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=i.label))

    def generate():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        generate,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

# Définition des colonnes de données et d'étiquettes
DATA_COLUMN = 'review'
LABEL_COLUMN = 'sentiment'

#### Conversion des données en objets TF

In [42]:

train_data = convertexamples2tf(list(train_examples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

  0%|          | 0/35000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 35000/35000 [03:31<00:00, 165.31it/s]


In [43]:
validation_data = convertexamples2tf(list(validation_examples), tokenizer)
validation_data = validation_data.batch(32)

100%|██████████| 35000/35000 [03:04<00:00, 189.32it/s]


In [46]:
test_data = convertexamples2tf(list(test_examples), tokenizer)
test_data = test_data.batch(32)

  0%|          | 0/35000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 35000/35000 [03:07<00:00, 187.08it/s]


In [45]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
2188/2188 [==============================] - 2261s 1s/step - loss: 0.2482 - accuracy: 0.8944 - val_loss: 0.0870 - val_accuracy: 0.9743
Epoch 2/2
2188/2188 [==============================] - 2180s 996ms/step - loss: 0.0678 - accuracy: 0.9762 - val_loss: 0.0411 - val_accuracy: 0.9863


In [47]:
# Évaluation finale sur l'ensemble de test
# test_data = convertexamples2tf(test_examples, tokenizer)
evaluation = model.evaluate(test_data)
print("Perte (loss) sur les données de test :", evaluation[0])
print("Précision (accuracy) sur les données de test :", evaluation[1])


1094/1094 [==============================] - 314s 287ms/step - loss: 0.0411 - accuracy: 0.9863
Perte (loss) sur les données de test : 0.04111315310001373
Précision (accuracy) sur les données de test : 0.9863428473472595


In [48]:
sentences = ['This was a good movie. I would watch it again','I cannot believe I have wasted time on this movie, it is the worst movie I have ever seen']
tf_batch = tokenizer(sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')   # we are tokenizing before sending into our trained model
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)       # axis=-1, this means that the index that will be returned by argmax will be taken from the *last* axis.
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(sentences)):
    print(sentences[i], ": ", labels[label[i]])

This was a good movie. I would watch it again :  Positive
I cannot believe I have wasted time on this movie, it is the worst movie I have ever seen :  Negative


In [49]:
test

,review,sentiment
47292,CORRIDORS OF BLOOD <br /><br />Aspect ratio: 1...,0
19353,I was pretty surprised with this flick. Even t...,1
8133,"This movie was really bad, plain and simple. H...",0
45868,"It's not funny, it's not interesting, it's not...",0
1697,I saw this recently on a cable channel. The mo...,1
...,...,...
6420,"Unless you're twelve, this movie really isn't ...",0
33928,Just an hour ago I finished watching this my f...,0
48421,I can't tell you all how much I love this movi...,1
26037,The Last Hunt is one of the few westerns ever ...,1


##LSTM

In [1]:
import os,re
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
print('tensorflow version:',tf.__version__)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, InputLayer, Dense, Embedding, Input
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt

tensorflow version: 2.15.0


In [2]:
df.head(3)

NameError: name 'df' is not defined